## Combining GFS Weather & Production Values 

In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")
pd.options.display.max_columns = None

In [42]:
weather = pd.read_parquet("../data/raw/weather_large.parquet")
production = pd.read_parquet("../data/raw/production_large.parquet")

farms = pd.read_csv("../data/meta/ege_farms.csv")

In [43]:
longitudes = weather.lon.unique()
latitudes = weather.lat.unique()

def generate_bbox_weather(lat_, lon_):
    lon_bounds = np.max(longitudes[longitudes < lon_]), np.min(longitudes[longitudes > lon_])
    lat_bounds = np.max(latitudes[latitudes < lat_]), np.min(latitudes[latitudes > lat_])
    # bbox = list(itertools.product(lon_bounds, lat_bounds))
    temp_weather = weather[(weather["lat"].isin(lat_bounds)) & (weather["lon"].isin(lon_bounds))].sort_values(["forecast_epoch", "lon", "lat"]).reset_index(drop=True)
    temp_weather = temp_weather.pivot_table(index="forecast_epoch", columns=["lon", "lat"], values=["UGRD_80.m.above.ground", "VGRD_80.m.above.ground"])
    # temp_weather.columns = temp_weather.columns.to_flat_index()
    temp_weather.columns = temp_weather.columns.get_level_values(0) + [".SW", ".NW", ".NE", ".SE"] * 2
    return temp_weather

In [44]:
df_list = []
for _, (lat, lon, plant) in farms[["lat", "lon", "rt_plant_id"]].iterrows():
    df_ = generate_bbox_weather(lat, lon)
    df_["rt_plant_id"] = plant
    df_list.append(df_)
    
weather_prepared = pd.concat(df_list).reset_index()
weather_prepared["rt_plant_id"] = weather_prepared["rt_plant_id"].astype(int)
weather_prepared.rename(columns={"forecast_epoch": "epoch"}, inplace=True)

In [45]:
def convert_epoch_column_to_datetime(epoch_column, turkey_time=True):
    """
    Convert epoch time to datetime object
    """
    utc = pd.to_datetime(epoch_column, unit="s").dt.tz_localize('UTC')
    if turkey_time:
        return utc.dt.tz_convert('Europe/Istanbul')
    return utc

In [46]:
production = production[production.rt_plant_id.isin(farms.rt_plant_id)]
production["forecast_dt"] = convert_epoch_column_to_datetime(production["epoch"], turkey_time=True)
production = production[production.forecast_dt.dt.year >= 2018].reset_index(drop=True)

In [47]:
production = pd.merge(production, weather_prepared, on=["epoch", "rt_plant_id"])

In [51]:
production.head()

,epoch,production,rt_plant_id,forecast_dt,UGRD_80.m.above.ground.SW,UGRD_80.m.above.ground.NW,UGRD_80.m.above.ground.NE,UGRD_80.m.above.ground.SE,VGRD_80.m.above.ground.SW,VGRD_80.m.above.ground.NW,VGRD_80.m.above.ground.NE,VGRD_80.m.above.ground.SE
0,1514754000,0.0,1503,2018-01-01 00:00:00+03:00,-0.842773,0.067227,-0.782773,0.067227,1.08792,2.18792,0.997922,2.03792
1,1514764800,0.0,1503,2018-01-01 03:00:00+03:00,-0.469976,0.720024,-0.179976,0.730024,1.73287,2.92287,1.242870,2.56287
2,1514775600,0.4,1503,2018-01-01 06:00:00+03:00,-0.932734,0.387266,-0.372734,0.477266,2.52399,3.67399,1.713990,3.02399
3,1514786400,2.0,1503,2018-01-01 09:00:00+03:00,-1.308400,-0.028396,-0.458396,0.201604,3.69407,4.74407,2.504070,3.39407
4,1514797200,2.0,1503,2018-01-01 12:00:00+03:00,-0.004195,1.555810,0.825805,1.795810,4.59009,5.41009,2.890090,3.56009


In [52]:
production.to_parquet("../data/processed/enriched.parquet")

## Missing Dates & Outlier Removed

93 plant için data 2018-01-01den başlıyor, kalanlar şöyle:
- 2288  &emsp;  2018-03-20
- 2291  &emsp;  2018-04-13
- 2323  &emsp;  2018-09-24
- 2326  &emsp;  2018-09-28
- 2374  &emsp;  2018-11-13
- 2397  &emsp;  2019-03-14
- 2420  &emsp;  2019-06-28
- 2538  &emsp;  2020-03-28

1 saatlik data 3 plant hariç hepsinde data 2019-01-21'de başlıyor, üç plant'in öncesinde 3 saatlik dataları da yok zaten
- 2397  &emsp;  2019-03-14
- 2420  &emsp;  2019-06-28
- 2538  &emsp;  2020-03-28

3 saatlik dönemde (2019-01-21 öncesi) :
- 6 saat fark olan bir gün var: 2018-05-27 06:00'de 

1 saatlik dönemde (2019-01-21 sonrası) :
- 3 saat fark olan 2019-01-24 00:00 ve 2019-01-24 03:00
- 2 saat fark olan 2019-04-15 21:00, 2019-05-05 00:00, 2019-08-17 04:00, 2021-06-08 08:00 ve 2021-06-09 20:00 

datayı 2019-01-24ten sonrasına filtrelersek 3 plant sonradan dahil oluyor olacak, 5 saat için de missing data oluyor

In [4]:
df = pd.read_parquet("../data/processed/enriched.parquet")
df = df[df["forecast_dt"] >= "2019-01-24 03:00:00+03:00"]

### Interpolation

In [5]:
df_ = df.sort_values(["rt_plant_id", "forecast_dt"]).copy()
df_["shifted"] = df_.groupby("rt_plant_id").forecast_dt.shift(1)
df_["diff"] = (df_.forecast_dt - df_.shifted)
df_[df_["diff"] != pd.Timedelta(hours=1)].groupby(["forecast_dt", "diff"], as_index=0).epoch.count()

,forecast_dt,diff,epoch
0,2019-04-15 21:00:00+03:00,0 days 02:00:00,94
1,2019-05-05 00:00:00+03:00,0 days 02:00:00,94
2,2019-08-17 04:00:00+03:00,0 days 02:00:00,94
3,2021-06-08 08:00:00+03:00,0 days 02:00:00,94
4,2021-06-09 20:00:00+03:00,0 days 02:00:00,94


In [6]:
weather_cols = [col for col in df.columns if col.startswith(("UGRD", "VGRD"))]

df.forecast_dt = df.forecast_dt.dt.tz_localize(None)
df = df.set_index("forecast_dt")[["rt_plant_id", "production", *weather_cols]]

df = df.groupby("rt_plant_id").resample("1H").interpolate()
df = df.drop(columns=["rt_plant_id"]).reset_index().set_index("forecast_dt").sort_index()
df = df.sort_values(["rt_plant_id", "forecast_dt"]).reset_index()

### Outlier Clipping

In [7]:
def detect_outlier(df, lower=0.01, upper=0.99, rolling=False):
    if rolling:
        q_lower = df.groupby("rt_plant_id").production.rolling(90 * 24).quantile(lower)
        q_upper = df.groupby("rt_plant_id").production.rolling(90 * 24).quantile(upper)
        df["production_lower"] = q_lower.values
        df["production_upper"] = q_upper.values
    else:
        q_lower = df.groupby("rt_plant_id").production.quantile(lower)
        q_upper = df.groupby("rt_plant_id").production.quantile(upper)
        q_lower.name = "production_lower"
        q_upper.name = "production_upper" 
        df = pd.merge(df, q_lower, on="rt_plant_id", how="left")
        df = pd.merge(df, q_upper, on="rt_plant_id", how="left")
    return df

def clip_outlier(df):
    assert "production_lower" in df.columns
    assert "production_upper" in df.columns
    df.loc[:, "is_outlier"] = (df.production < df.production_lower) | (df.production > df.production_upper)
    df["production_cleaned"] = np.clip(df.production, df.production_lower, df.production_upper)
    return df

In [8]:
df = detect_outlier(df, 0.01, 0.99, rolling=False)
df = clip_outlier(df)

df = df.rename(columns={"production": "production_old"})
df = df.rename(columns={"production_cleaned": "production"})

In [10]:
# df[df["is_outlier"]].groupby("rt_plant_id").production.count().sort_values().tail(10)
# df[df["rt_plant_id"] == 1487].set_index("forecast_dt")[["production", "production_lower", "production_upper"]].plot(figsize=(20, 10))

# azalan 2062 1504
# artan 1524 1523 1939 1944
# artıp azalan 1939 1781
# garip artan 969
# garip azalan 1878
# stabil 1787
# net outlier 1488 2050

# capacity & outlier karışık 1761

In [14]:
df.to_parquet("../data/processed/outlier_removed.parquet")